In [41]:
import pandas as pd
import numpy as np
import seaborn as np
from matplotlib import pyplot as plt
import re

In [42]:
def extract_measurement(b_list):

    # this function extract the data points from B3,B4,B5 and 
    # And return clean data to get_measurment() in dictionay format 
    
    x_spos = b_list.index('LengthValue') # starting index of the x values
    y_spos = b_list.index('Value')# starting index of the y values
    y_epos = len(b_list)# the ending pos of y values
    
    x = b_list[x_spos + 1:y_spos-1]
    y = b_list[y_spos + 1: y_epos -2]

    null_index = set() # a set to keep the index of null values in either x or y
    
    # loop over x and get the index of the 'null' elements
    for index,value in enumerate(x):
        if value == "null":
            null_index.add(index)

    # loop over y and get the index of the 'null' elements
    for index,value in enumerate(y):
        if value == "null":
            null_index.add(index)    
    

    # removing the x,y pair with null value in either of them
    x = [value for idx, value in enumerate(x) if idx not in null_index]
    y = [value for idx, value in enumerate(y) if idx not in null_index]

    # changing the elements of x and y from string to float
    x_float =[float(item) for item in x]
    y_float =[float(item) for item in y]

    xy_dict = dict(zip(x_float,y_float)) # zip x and y  in dictionary
    print(len(xy_dict))
    #print(xy_dict)

    
    


    return xy_dict




def get_measurment(coil_id):

    # this function load the data for B3,B4,B5 measurments
    # It split the data based on ";"
    # It calls extract_measuremnt() function to extract the 
    # clean data pairs for B3,B4,B5
    # And return the values in dictionary format

    print('************************************')
    print("coil Id: ", coil_id)
    root_directory = "data/SampleProcessedData/"

    b3_df = pd.read_csv(root_directory + str(coil_id ) + "B3.csv")
    b4_df = pd.read_csv(root_directory + str(coil_id ) + "B4.csv")
    b5_df = pd.read_csv(root_directory + str(coil_id ) + "B5.csv")

    b3_list = re.split(';', str(b3_df.columns))
    b4_list = re.split(';', str(b4_df.columns))
    b5_list = re.split(';', str(b5_df.columns))

    b_lists = [b3_list,b4_list,b5_list]

    all_B_list = [] # this list contain all xy_dict for B3,B4,B5(it is list of dictionaries)
    for b_list in b_lists:
        all_B_list.append(extract_measurement(b_list))
        
    
    print('*********************************')


    
    return all_B_list # return list of dictionaries
    



    
 
    

    

In [43]:
def label_observation():

    # this function get the coil_Id and call the get_measurment()
    # function to get the cleaned value for B3,B4 and B5 in dictionary format
    # from the dictionary it checks if there is constriction or not
    # and add the result to the coil_df

     #load the dataset
     coil_df = pd.read_excel('data/CoilDataInput.xlsx')
     count = 0

     measurment = get_measurment(290044)
     print(measurment)


     # for name,coil_id in coil_df['coil'].items():
     #      try:
     #           get_measurment(coil_id)
     #      except:
     #           print('not found')
     #           continue
          
        


     

label_observation()
     


     


************************************
coil Id:  290044
1055
1054
1031
*********************************
[{0.0: 1491.8, 0.4151917: 1490.7, 0.8307152: 1489.6, 1.245243: 1488.0, 1.659551: 1485.6, 2.073751: 1483.8, 2.487951: 1483.0, 2.902592: 1482.9, 3.317011: 1482.9, 3.731651: 1482.6, 4.146292: 1482.2, 4.560932: 1481.5, 4.975792: 1481.1, 5.391205: 1480.5, 5.806728: 1480.1, 6.222252: 1480.2, 6.638103: 1480.6, 7.054951: 1480.5, 7.471798: 1480.1, 7.889526: 1480.1, 8.306482: 1480.4, 8.723438: 1480.5, 9.140285: 1480.2, 9.557354: 1480.2, 9.974754: 1480.0, 10.39182: 1480.1, 10.80933: 1480.5, 11.22651: 1480.5, 11.64412: 1481.0, 12.06196: 1481.6, 12.47969: 1482.0, 12.89764: 1481.5, 13.31537: 1480.9, 13.73332: 1480.8, 14.15205: 1481.3, 14.57121: 1481.4, 14.99093: 1481.4, 15.41086: 1481.1, 15.83014: 1480.9, 16.2492: 1481.0, 16.66869: 1481.1, 17.08863: 1480.5, 17.50901: 1480.9, 17.92906: 1481.1, 18.34976: 1481.3, 18.77003: 1481.1, 19.19118: 1480.4, 19.61255: 1480.3, 20.03393: 1480.1, 20.4553: 1480.4, 

In [ ]:
def visualize_measurments():
    pass